# Demo of `columnsfmri` 
## Simulation and optimization of fMRI of cortical columns

Import model implementation from columnsfmri.py and other useful modules.

In [ ]:
import columnsfmri

%matplotlib inline
import numpy as np
import importlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Inititialize simulation using a 512 x 512 grid on an area of 24 x 24 mm. 

In [ ]:
N = 512; L = 24
sim = columnsfmri.simulation(N,L)

Simulate a column pattern by filtering Gaussian white noise. Rho is the main pattern frequency, delta specifies the amount of irregularity.

In [ ]:
gwn = sim.gwnoise()
rho,deltaRelative = 0.2,1
columnPattern = sim.columnPattern(rho,deltaRelative,gwn)
sim.plotPattern(columnPattern)

Simulate a spatial BOLD response with a FWHM of 2 mm, and an average response amplitude of 5%.

In [ ]:
fwhm = 2
beta = 0.05
boldPattern,_,_ = sim.bold(fwhm,beta,columnPattern)
sim.plotPattern(boldPattern)

Simulate MRI sampling using a voxel width of 3 mm. (We first add the relative response pattern to a constant background of 1).

In [ ]:
w = 1
mriPattern = sim.mri(w,1+boldPattern)
sim.plotVoxels(mriPattern)

The amount of functional contrast can be quantified as the standard deviation of the imaged responses (contrast range).

In [ ]:
c = np.std(mriPattern)
print(str(100*c)+"%")

Simulate the noise level as a function of voxel width.

In [ ]:
w = np.linspace(0.1,3,100)
V = w**3
TR = 2
nT = 100
differentialFlag = True
noiseType = '3T'
SNR = 1/columnsfmri.noiseModel(V,TR,nT,differentialFlag,noiseType=noiseType)

In [ ]:
plt.plot(w,SNR)
plt.xlabel('voxel width [mm]')
plt.ylabel('multi measurement SNR')
plt.title('3T, TR = 2s, nT = 100')
plt.show()

Calculate detection probability from CNR and number of voxels.

In [ ]:
CNR = 1
nVoxels = 10
columnsfmri.detectionProbability(CNR,nVoxels)

Calculate the correlation between the original and the (interpolated) imaged pattern.

In [ ]:
sim.patternCorrelation(columnPattern,mriPattern)

Show column pattern, bold response and fMRI image next to each other.

In [ ]:
sim.plotColumnsBoldMRI(columnPattern,boldPattern,mriPattern)

In [ ]:
from ipywidgets import interactive, interact, interact_manual

N = 512; L = 24
s = columnsfmri.simulation(N,L)
gwn = sim.gwnoise()
rho,deltaRelative = 0.5,0.5
fwhm = 2
beta = 0.05
w = 1

def f(rho,deltaRelative,fwhm,w):
    columnPattern = sim.columnPattern(rho,deltaRelative,gwn)
    boldPattern,_,_ = sim.bold(fwhm,beta,columnPattern)
    mriPattern = sim.mri(w,1+boldPattern)
    sim.plotColumnsBoldMRI(columnPattern,boldPattern,mriPattern)

In [ ]:
interact(f,rho=[0.1,0.2,0.4,0.8,1.6,3.2],
      deltaRelative=[0.01, 0.25, 0.5, 0.75, 1],
      fwhm=[0.01, 0.5, 1, 1.5, 2,2.5, 3, 3.5],
      w=[0.25,0.5,1,1.5,2,3]);

Set standard parameters for optimization simulation.

In [ ]:
parameters = columnsfmri.setParameters('irregular')
for parameter,value in parameters.items():
    print(parameter + ": " + str(value))

Run optimization simulation.

In [ ]:
results = columnsfmri.simulatefMRIOfColumnPatterns(parameters)

Calculate CNR and plot as a function of voxel width.

In [ ]:
# wRange = parameters['wRange']
# cnr = cr/noiseOfW
# cnrMean = np.mean(cnr,axis=0)
# cnrStd = np.std(cnr,axis=0)
# cnrErrorPlus = cnrMean+2*cnrStd
# cnrErrorMinus = cnrMean-2*cnrStd

In [ ]:
# plt.plot(wRange, cnrMean)
# plt.fill_between(wRange, cnrErrorMinus, cnrErrorPlus,alpha=0.5)
# plt.xlabel('voxel width [mm]')
# plt.ylabel('CNR')
# plt.show()

In [ ]:
columnsfmri.printResults(results)